## Performing Image Registration on segmentation masks and extracting transformmatrix


In [2]:
#%matplotlib notebook
import os
import pyelastix
from PIL import Image
import numpy as np 
import cv2
import json
import pydicom
import SimpleITK as sitk
from os import path
from matplotlib import pyplot as plt
from matplotlib import image as mpimg


input_path ='./Masks/'
ref_path = './Reference/Reference_Mask.png'

## Functions

In [3]:
# creates a new folder if it does not already exist
def new_dir(dirName):
    try:
        os.mkdir(dirName)
        print("Directory " , dirName ,  " Created ") 
    except FileExistsError:
        print("Directory " , dirName ,  " already exists")

# target and reference are assumed numpy arrays of image files of the same resolution
def registration(target, reference, t):
    
    #internal elastix parameters
    params = pyelastix.get_default_params(type=t) #RIGID, AFFINE, BSPLINE
    params.NumberOfResolutions = 8
    params.AutomaticTransformInitialization = True
    params.AutomaticScalesEstimation = False
    params.NumberOfHistogramBins = 64
    params.MaximumStepLength = 4.0
    params.MaximumNumberOfIterations = 500

    registered, field = pyelastix.register(target, reference, params, verbose=0)
    return registered, field

def new_registration(target, reference, t):
    
    fixedImage = sitk.ReadImage(reference)
    movingImage = sitk.ReadImage(target)
    parameterMap = sitk.GetDefaultParameterMap(t)

    elastixImageFilter = sitk.ElastixImageFilter()
    elastixImageFilter.SetFixedImage(fixedImage)
    elastixImageFilter.SetMovingImage(movingImage)
    elastixImageFilter.SetParameterMap(parameterMap)
    elastixImageFilter.Execute()
    
    resultImage = elastixImageFilter.GetResultImage()
    transformParameterMap = elastixImageFilter.GetTransformParameterMap()

    return resultImage, transformParameterMap
  

## Execution Pipeline 



### Affine Registrations

In [6]:
%%time
new_dir("Pngs")
new_dir("Registered_with_PyElastix")
new_dir("Registered_Masks")

reference = np.asarray(Image.open(ref_path)).astype(float)
filename = "test_file_1_Mask.png"
path = os.path.join(input_path, filename)
target = np.asarray(Image.open(path)).astype(float)
path2 = os.path.join('Registered_Masks', filename) 
r, f = registration(target, reference,"AFFINE")

#print(f.size)
print(f.shape)

ret, r = cv2.threshold(r, 120, 255, cv2.THRESH_BINARY)

plt.figure(figsize=(20, 18))
plt.subplot(3, 3, 1)
plt.title('Reference')
plt.imshow(reference, cmap='gray')
plt.subplot(3, 3, 2)
plt.title('Moving Image')
plt.imshow(target, cmap='gray')
plt.subplot(3, 3, 3)
plt.title('Registered Mask')
plt.imshow(r, cmap='gray')
result = Image.fromarray(r)
result.convert('P').save(path2.replace('.png','_AFFINE.png'), 'PNG')
    

Directory  Pngs  already exists
Directory  Registered_with_PyElastix  already exists
Directory  Registered_Masks  already exists


AttributeError: 'tuple' object has no attribute 'shape'

In [10]:
print(f[1][5])




[-4.3699784 -4.3495226 -4.3290663 -4.3086104 -4.2881546 -4.2676983
 -4.2472425 -4.2267866 -4.2063303 -4.1858745 -4.1654186 -4.1449623
 -4.1245065 -4.1040506 -4.0835943 -4.0631385 -4.0426826 -4.0222263
 -4.0017705 -3.9813144 -3.9608586 -3.9404025 -3.9199464 -3.8994906
 -3.8790345 -3.8585784 -3.8381226 -3.8176665 -3.7972105 -3.7767546
 -3.7562985 -3.7358425 -3.7153864 -3.6949306 -3.6744745 -3.6540184
 -3.6335626 -3.6131065 -3.5926504 -3.5721946 -3.5517385 -3.5312824
 -3.5108266 -3.4903705 -3.4699144 -3.4494586 -3.4290025 -3.4085464
 -3.3880906 -3.3676345 -3.3471785 -3.3267224 -3.3062665 -3.2858105
 -3.2653544 -3.2448986 -3.2244425 -3.2039864 -3.1835306 -3.1630745
 -3.1426184 -3.1221626 -3.1017065 -3.0812504 -3.0607946 -3.0403385
 -3.0198824 -2.9994266 -2.9789705 -2.9585145 -2.9380586 -2.9176025
 -2.8971465 -2.8766904 -2.8562346 -2.8357785 -2.8153224 -2.7948666
 -2.7744105 -2.7539544 -2.7334986 -2.7130425 -2.6925864 -2.6721306
 -2.6516745 -2.6312184 -2.6107626 -2.5903065 -2.5698504 -2.549

In [5]:
%%time
new_dir("Pngs")
new_dir("Registered_with_PyElastix")
new_dir("Registered_Masks")

#reference = np.asarray(Image.open(ref_path)).astype(float)
filename = "test_file_1_Mask.png"
path = os.path.join(input_path, filename)
#target = np.asarray(Image.open(path)).astype(float)
path2 = os.path.join('Registered_Masks', filename) 

#new_r = new_registration(path, ref_path,"AFFINE")

fixedImage = sitk.ReadImage(ref_path)
movingImage = sitk.ReadImage(path)
parameterMap = sitk.GetDefaultParameterMap("Affine")

elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.SetFixedImage(fixedImage)
elastixImageFilter.SetMovingImage(movingImage)
elastixImageFilter.SetParameterMap(parameterMap)
elastixImageFilter.Execute()

resultImage = elastixImageFilter.GetResultImage()
transformParameterMap = elastixImageFilter.GetTransformParameterMap()

ret, new_r = cv2.threshold(new_r, 120, 255, cv2.THRESH_BINARY)


plt.figure(figsize=(20, 18))
plt.subplot(3, 3, 1)
plt.title('Reference')
plt.imshow(reference, cmap='gray')
plt.subplot(3, 3, 2)
plt.title('Moving Image')
plt.imshow(target, cmap='gray')
plt.subplot(3, 3, 3)
plt.title('Registered Mask')
plt.imshow(new_r, cmap='gray')
result = Image.fromarray(r)
result.convert('P').save(path2.replace('.png','_AFFINE_ITK.png'), 'PNG')
    

Directory  Pngs  already exists
Directory  Registered_with_PyElastix  already exists
Directory  Registered_Masks  already exists


AttributeError: module 'SimpleITK' has no attribute 'GetDefaultParameterMap'

## Observations